# 株価取得処理

In [1]:
# ライブラリのインストール
!pip install yfinance


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import yfinance as yf
import pandas as pd

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# トヨタ自動車 (7203.T) のデータを取得
ticker_code = '7203.T'
ticker = yf.Ticker(ticker_code)

# 1. 株価データの取得 (過去1年分の日足)
print("--- 1. 株価データ (過去5日分) ---")
hist = ticker.history(period="1y")
print(hist.tail()) # .tail()で末尾5日分を表示

print("\n" + "="*50 + "\n")

# 2. 企業情報・指標の取得
print("--- 2. 企業情報・指標 ---")
info = ticker.info

# 必要な情報を抽出して表示
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice')}")
print(f"PER (株価収益率): {info.get('trailingPE')}")
print(f"PBR (株価純資産倍率): {info.get('priceToBook')}")
print(f"配当利回り: {info.get('dividendYield')}")

# 3. 配当性向の取得
# infoの中に'payoutRatio'というキーで配当性向が含まれている
print("\n--- 3. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    print(f"配当性向: {payout_ratio:.2%}") # パーセント表示
else:
    print("配当性向データがありません。")

--- 1. 株価データ (過去5日分) ---
                             Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                                        
2025-06-09 00:00:00+09:00  2670.0  2672.0  2637.0  2640.0  13584200        0.0           0.0
2025-06-10 00:00:00+09:00  2653.0  2663.5  2640.0  2657.5  19556800        0.0           0.0
2025-06-11 00:00:00+09:00  2670.0  2678.0  2652.0  2656.0  16367500        0.0           0.0
2025-06-12 00:00:00+09:00  2645.0  2648.0  2608.0  2616.5  22784400        0.0           0.0
2025-06-13 00:00:00+09:00  2586.0  2589.0  2539.0  2555.0  36054800        0.0           0.0


--- 2. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
株価: 2555.0
PER (株価収益率): 7.107291
PBR (株価純資産倍率): 0.9280495
配当利回り: 3.72

--- 3. 配当性向 ---
配当性向: 25.03%


In [11]:
import yfinance as yf
import pandas as pd
import numpy as np # RCIの計算に必要

# --- 関数定義 ---

def add_rci(df, period):
    """
    DataFrameにRCI（順位相関指数）の列を追加する関数
    
    Args:
        df (pd.DataFrame): 'Close'列を含む株価データ
        period (int): RCIの計算期間
        
    Returns:
        pd.DataFrame: RCI列が追加されたDataFrame
    """
    # RCIを計算するための内部関数
    def rci_calc(series):
        n = len(series)
        # 日付の順位は 1, 2, ..., n
        date_rank = pd.Series(range(1, n + 1))
        # 価格の順位
        price_rank = pd.Series(series).rank(method='first')
        # d (順位の差の二乗和) を計算
        d = ((date_rank - price_rank.reset_index(drop=True)) ** 2).sum()
        # RCIを計算
        return (1 - (6 * d) / (n * (n**2 - 1))) * 100

    column_name = f'RCI_{period}'
    # rolling().apply() を使って各期間でRCIを計算
    df[column_name] = df['Close'].rolling(window=period).apply(rci_calc, raw=True)
    return df

def calculate_fibonacci(df):
    """
    フィボナッチ・リトレースメントのレベルを計算する関数
    
    Args:
        df (pd.DataFrame): 'High'と'Low'列を含む株価データ
        
    Returns:
        tuple: (フィボナッチレベルの辞書, 期間最高値, 期間最安値)
    """
    # 期間内の最高値と最安値を取得
    high_price = df['High'].max()
    low_price = df['Low'].min()
    
    # 値幅
    diff = high_price - low_price
    
    # フィボナッチ比率
    ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    
    levels = {}
    print(f"フィボナッチ・リトレースメント (押し目・サポートの目安)")
    print(f"期間最高値: {high_price:,.2f} 円")
    print(f"期間最安値: {low_price:,.2f} 円")
    print("-" * 30)

    # 各レベルの価格を計算（高値からの下落率で計算）
    for ratio in sorted(ratios, reverse=True):
        level_price = high_price - diff * ratio
        levels[f'{ratio*100:.1f}%'] = level_price
        
    return levels

# --- メイン処理 ---

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# 銘柄コードとデータ取得期間
ticker_code = '7203.T' # トヨタ自動車
period_to_get = "1y"   # 過去1年分

# Tickerオブジェクトを作成
ticker = yf.Ticker(ticker_code)

# 1. 企業情報の取得
print("--- 1. 企業情報・指標 ---")
info = ticker.info
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice', 'N/A'):,.2f} 円")
print(f"PER (株価収益率): {info.get('trailingPE', 'N/A'):.2f} 倍")
print(f"PBR (株価純資産倍率): {info.get('priceToBook', 'N/A'):.2f} 倍")

# 配当利回りはパーセント表示に修正
div_yield = info.get('dividendYield')
if div_yield:
    print(f"配当利回り: {div_yield:.2%}")
else:
    print(f"配当利回り: N/A")

print("\n" + "="*50 + "\n")


# 2. 株価データとRCIの取得・計算
print(f"--- 2. 最新の株価とRCI (対象期間: {period_to_get}) ---")
hist = ticker.history(period=period_to_get)

# 短期・中期・長期のRCIを計算してDataFrameに追加
hist = add_rci(hist, 9)
hist = add_rci(hist, 26)
hist = add_rci(hist, 52)

# 最新5日間の終値とRCIを表示
print(hist[['Close', 'RCI_9', 'RCI_26', 'RCI_52']].tail())

print("\n" + "="*50 + "\n")


# 3. フィボナッチ・リトレースメントの計算と表示
print(f"--- 3. フィボナッチライン (対象期間: {period_to_get}) ---")
fib_levels = calculate_fibonacci(hist)

# 計算結果を整形して表示
for level_name, level_price in fib_levels.items():
    print(f"  {level_name.rjust(6)}: {level_price:,.2f} 円")

print("\n" + "="*50 + "\n")


# 4. 配当性向の取得
print("--- 4. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    # 0~1の範囲で返ってくるのでパーセント表示
    print(f"配当性向: {payout_ratio:.2%}")
else:
    print("配当性向データがありません。")

--- 1. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
株価: 2,539.50 円
PER (株価収益率): 7.06 倍
PBR (株価純資産倍率): 0.92 倍
配当利回り: 372.00%


--- 2. 最新の株価とRCI (対象期間: 1y) ---
                            Close      RCI_9     RCI_26     RCI_52
Date                                                              
2025-06-11 00:00:00+09:00  2656.0 -68.333333 -30.871795  39.417741
2025-06-12 00:00:00+09:00  2616.5 -68.333333 -35.452991  44.309741
2025-06-13 00:00:00+09:00  2555.0 -71.666667 -42.495726  46.222146
2025-06-16 00:00:00+09:00  2554.5 -73.333333 -46.119658  41.697259
2025-06-17 00:00:00+09:00  2539.5 -73.333333 -47.145299  38.188338


--- 3. フィボナッチライン (対象期間: 1y) ---
フィボナッチ・リトレースメント (押し目・サポートの目安)
期間最高値: 3,289.93 円
期間最安値: 2,112.95 円
------------------------------
  100.0%: 2,112.95 円
   78.6%: 2,364.82 円
   61.8%: 2,562.56 円
   50.0%: 2,701.44 円
   38.2%: 2,840.32 円
   23.6%: 3,012.16 円
    0.0%: 3,289.93 円


--- 4. 配当性向 ---
配当性向: 25.03%


In [5]:
pip install --upgrade yfinance



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import yfinance as yf
import pandas as pd
import numpy as np # RCIの計算に必要

# --- 関数定義 ---

def add_rci(df, period):
    """
    DataFrameにRCI（順位相関指数）の列を追加する関数
    
    Args:
        df (pd.DataFrame): 'Close'列を含む株価データ
        period (int): RCIの計算期間
        
    Returns:
        pd.DataFrame: RCI列が追加されたDataFrame
    """
    # RCIを計算するための内部関数
    def rci_calc(series):
        n = len(series)
        # 日付の順位は 1, 2, ..., n
        date_rank = pd.Series(range(1, n + 1))
        # 価格の順位
        price_rank = pd.Series(series).rank(method='first')
        # d (順位の差の二乗和) を計算
        d = ((date_rank - price_rank.reset_index(drop=True)) ** 2).sum()
        # RCIを計算
        return (1 - (6 * d) / (n * (n**2 - 1))) * 100

    column_name = f'RCI_{period}'
    # rolling().apply() を使って各期間でRCIを計算
    df[column_name] = df['Close'].rolling(window=period).apply(rci_calc, raw=True)
    return df

def calculate_fibonacci(df):
    """
    フィボナッチ・リトレースメントのレベルを計算する関数
    
    Args:
        df (pd.DataFrame): 'High'と'Low'列を含む株価データ
        
    Returns:
        tuple: (フィボナッチレベルの辞書, 期間最高値, 期間最安値)
    """
    # 期間内の最高値と最安値を取得
    high_price = df['High'].max()
    low_price = df['Low'].min()
    
    # 値幅
    diff = high_price - low_price
    
    # フィボナッチ比率
    ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    
    levels = {}
    print(f"フィボナッチ・リトレースメント (押し目・サポートの目安)")
    print(f"期間最高値: {high_price:,.2f} 円")
    print(f"期間最安値: {low_price:,.2f} 円")
    print("-" * 30)

    # 各レベルの価格を計算（高値からの下落率で計算）
    for ratio in sorted(ratios, reverse=True):
        level_price = high_price - diff * ratio
        levels[f'{ratio*100:.1f}%'] = level_price
        
    return levels

# --- メイン処理 ---

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# 銘柄コードとデータ取得期間
ticker_code = '7203.T' # トヨタ自動車
period_to_get = "1y"   # 過去1年分

# Tickerオブジェクトを作成
ticker = yf.Ticker(ticker_code)

# 1. 企業情報の取得
print("--- 1. 企業情報・指標 ---")
info = ticker.info
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice', 'N/A'):,.2f} 円")
print(f"PER (株価収益率): {info.get('trailingPE', 'N/A'):.2f} 倍")
print(f"PBR (株価純資産倍率): {info.get('priceToBook', 'N/A'):.2f} 倍")

# 配当利回りはパーセント表示に修正
div_yield = info.get('dividendYield')
if div_yield:
    print(f"配当利回り: {div_yield:.2%}")
else:
    print(f"配当利回り: N/A")

print("\n" + "="*50 + "\n")


# 2. 株価データとRCIの取得・計算
print(f"--- 2. 最新の株価とRCI (対象期間: {period_to_get}) ---")
hist = ticker.history(period=period_to_get)

# 短期・中期・長期のRCIを計算してDataFrameに追加
hist = add_rci(hist, 9)
hist = add_rci(hist, 26)
hist = add_rci(hist, 52)

# 最新5日間の終値とRCIを表示
print(hist[['Close', 'RCI_9', 'RCI_26', 'RCI_52']].tail())

print("\n" + "="*50 + "\n")


# 3. フィボナッチ・リトレースメントの計算と表示
print(f"--- 3. フィボナッチライン (対象期間: {period_to_get}) ---")
fib_levels = calculate_fibonacci(hist)

# 計算結果を整形して表示
for level_name, level_price in fib_levels.items():
    print(f"  {level_name.rjust(6)}: {level_price:,.2f} 円")

print("\n" + "="*50 + "\n")


# 4. 配当性向の取得
print("--- 4. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    # 0~1の範囲で返ってくるのでパーセント表示
    print(f"配当性向: {payout_ratio:.2%}")
else:
    print("配当性向データがありません。")

--- 1. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
株価: 2,538.50 円
PER (株価収益率): 7.06 倍
PBR (株価純資産倍率): 0.92 倍
配当利回り: 372.00%


--- 2. 最新の株価とRCI (対象期間: 1y) ---
                            Close      RCI_9     RCI_26     RCI_52
Date                                                              
2025-06-11 00:00:00+09:00  2656.0 -68.333333 -30.871795  39.417741
2025-06-12 00:00:00+09:00  2616.5 -68.333333 -35.452991  44.309741
2025-06-13 00:00:00+09:00  2555.0 -71.666667 -42.495726  46.222146
2025-06-16 00:00:00+09:00  2554.5 -73.333333 -46.119658  41.697259
2025-06-17 00:00:00+09:00  2538.5 -73.333333 -47.145299  38.188338


--- 3. フィボナッチライン (対象期間: 1y) ---
フィボナッチ・リトレースメント (押し目・サポートの目安)
期間最高値: 3,289.93 円
期間最安値: 2,112.95 円
------------------------------
  100.0%: 2,112.95 円
   78.6%: 2,364.82 円
   61.8%: 2,562.56 円
   50.0%: 2,701.44 円
   38.2%: 2,840.32 円
   23.6%: 3,012.16 円
    0.0%: 3,289.93 円


--- 4. 配当性向 ---
配当性向: 25.03%
